In [1]:
import pandas as pd
import numpy as np
#import featuretools as ft
import calendar
#import xmltodict

In [2]:
#
# справочник выходных и праздничных дней с 1999 до 2025 гг.

work_days = pd.read_csv('work.csv', encoding='1251', index_col='Год/Месяц')

# словарь национальный праздников РФ

dict_hol = {101: 'НГ', 102: 'НГ', 103: 'НГ', 104: 'НГ', 105: 'НГ', 106: 'НГ',
     107: 'Рожд', 223: '23фев', 308: '8мар', 501: '1мая', 509: '9мая', 612: '12ию', 114: '4ноя'}

In [3]:
#
# признаки из дейттайм

def datefeatures(df, i = 'index', date = 0, time = 1, dist = 1):
    
    # признаки из дат
    
    if date == 0:
    
        # номер года

        dfd[i + '_year'] = df.dt.year

        # номер месяца

        dfd[i + '_mnth'] = df.dt.month

        # номер недели

        dfd[i + '_week'] = df.dt.week 

        # 1 декада?

        dfd[i + '_is_dcd1'] = df.map(lambda x: 1 if x.date().day < 11 else 0)

        # 2 декада?

        dfd[i + '_is_dcd2'] = df.map(lambda x: 1 if 10 < x.date().day < 21 else 0)

        # номер дня недели

        dfd[i + '_dow'] = df.apply(lambda x: x.date().weekday())

        # номер дня месяца (ВНИМАНИЕ! большая нагрузка на память при переходе к дамми)

        dfd[i + '_day'] = df.dt.day

        # сб или вс?

        dfd[i + '_is_eow'] = df.apply(lambda x: 1 if x.date().weekday() in (5, 6) else 0)
        
        # выходной (1999-2025 гг.)?
        
        try:
            dfd[i + '_is_wknd'] = dfd['datetime_0'].apply(lambda x: int(str(x.day) in work_days[str(x.month)][x.year].split(',')))
        except:
            pass
        
        # национальные праздники
        
        dfd[i + '_ruhol'] = dfd['datetime_0'].map(lambda x: dict_hol[x.month*100+x.day] if dict_hol.get(x.month*100+x.day) else 'нет')


        # при необходимости сохранить дистанции

        if dist == 1: 

            # день месяца в sin, cos

            dfd[i + '_day_cos'] = df.apply(lambda x: make_harmonic_features(x.day, \
                                    calendar.monthrange(x.year, x.month)[1])[0])
            dfd[i + '_day_sin'] = df.apply(lambda x: make_harmonic_features(x.day, \
                                    calendar.monthrange(x.year, x.month)[1])[1])

            # номер месяца в sin, cos

            dfd[i + '_mnth_cos'] = make_harmonic_features(df.dt.month, 12)[0]
            dfd[i + '_mnth_sin'] = make_harmonic_features(df.dt.month, 12)[1]

    
    
    # признаки из времени

    if time == 0:

        # час

        dfd[i + '_hr'] = df.dt.hour

        # минута 

        dfd[i + '_mnt'] = df.dt.minute

        # секунда 

        dfd[i + '_sec'] = df.dt.second
        

        # при необходимости сохранить дистанции
        
        if dist == 0:
            
            
            # час в sin, cos
            
            dfd[i + '_hr_cos'] = df.apply(lambda x: make_harmonic_features(x.hour, 24)[0])
            dfd[i + '_hr_sin'] = df.apply(lambda x: make_harmonic_features(x.hour, 24)[1])
            
            # минута в sin, cos
            
            dfd[i + '_mnt_cos'] = df.apply(lambda x: make_harmonic_features(x.minute, 60)[0])
            dfd[i + '_mnt_sin'] = df.apply(lambda x: make_harmonic_features(x.minute, 60)[1])

            
            

    return dfd[dfd.columns[1:]]
  
        
# эзотерический подход

def make_harmonic_features(value, period):
    
    value *= 2 * np.pi / period
    return np.cos(value), np.sin(value)



# на вход подается pandas.DataFrame с именем train

In [4]:
#train = pd.read_csv('data/check_1_r/train.csv', parse_dates=True)

#
# выделить столбцы с датами в отдельный датасет

dfd = train.filter(regex='date*').apply(pd.to_datetime)
#dfd.info()

In [5]:
#
# утилита работает постолбчато

dfd_feat = pd.DataFrame(index = dfd.index)

for feat in dfd.columns:
    df_temp = (datefeatures(dfd[feat], i = feat, \
                            date = (dfd[feat].iloc[0].date() == dfd[feat].iloc[-1].date() == pd.to_datetime('00:00:00').date()), \
                            time = (dfd[feat].iloc[0].time() == dfd[feat].iloc[-1].time() == pd.to_datetime('2000-01-01').time()), \
                            dist = 1))
    dfd_feat = dfd_feat.join(df_temp, how='outer')

In [6]:
dfd_feat.head()

,datetime_0_year,datetime_0_mnth,datetime_0_week,datetime_0_is_dcd1,datetime_0_is_dcd2,datetime_0_dow,datetime_0_day,datetime_0_is_eow,datetime_0_is_wknd,datetime_0_ruhol,datetime_0_day_cos,datetime_0_day_sin,datetime_0_mnth_cos,datetime_0_mnth_sin
0,2017,3,10,0,1,5,11,1,1,нет,-0.612106,0.790776,6.123234e-17,1.000000e+00
1,2017,1,2,1,0,1,10,0,0,нет,-0.440394,0.897805,8.660254e-01,5.000000e-01
2,2017,2,8,0,0,6,26,1,1,нет,0.900969,-0.433884,5.000000e-01,8.660254e-01
3,2017,2,5,1,0,3,2,0,0,нет,0.900969,0.433884,5.000000e-01,8.660254e-01
4,2017,6,23,1,0,3,8,0,0,нет,-0.104528,0.994522,-1.000000e+00,1.224647e-16
